# Import Libraries

In [ ]:
import numpy as np
from sklearn.cluster import KMeans



# **Part 1.1.1 — Generate the Ball & Beam Dataset**

In [ ]:

# Number of samples
T = 1000

# Initialize arrays
u = np.zeros(T)
y = np.zeros(T)

# Generate input signal: u(t) = sin(2πt / 250)
for t in range(T):
    u[t] = np.sin(2 * np.pi * t / 250)

# Initial conditions
y[0] = 0
y[1] = 0

# Generate system output y(t)
for t in range(2, T):
    y[t] = (y[t-1] / (1 + y[t-2])) + (u[t-1] / 3)

# Build dataset (X, Y)
# X contains: [y(t-1), y(t-2), u(t-1)]
# Y contains: y(t)

X = np.zeros((T-2, 3))
Y = np.zeros(T-2)

for t in range(2, T):
    X[t-2, :] = [y[t-1], y[t-2], u[t-1]]
    Y[t-2] = y[t]

print("X shape:", X.shape)
print("Y shape:", Y.shape)
print("First 5 samples:")
print(X[:5], "\nTargets:", Y[:5])


X shape: (998, 3)
Y shape: (998,)
First 5 samples:
[[0.         0.         0.0251301 ]
 [0.0083767  0.         0.05024432]
 [0.0251248  0.0083767  0.07532681]
 [0.05002503 0.0251248  0.10036171]
 [0.08225287 0.05002503 0.12533323]] 
Targets: [0.0083767  0.0251248  0.05002503 0.08225287 0.12011194]


# **Part 1.1.2 — Split Data into Training and Testing Sets**

In [ ]:
# Assuming you already generated X and Y in part 1.1.1

# Split sizes
train_size = 700
test_size = X.shape[0] - train_size  # Should be 298 (or 300 if X had 1000-2=998 samples)

# Training data
X_train = X[:train_size]
Y_train = Y[:train_size]

# Testing data
X_test = X[train_size:]
Y_test = Y[train_size:]

print("Training set shapes:", X_train.shape, Y_train.shape)
print("Testing set shapes:", X_test.shape, Y_test.shape)

print("\nExample training samples:\n", X_train[:3])
print("Example training targets:", Y_train[:3])


Training set shapes: (700, 3) (700,)
Testing set shapes: (298, 3) (298,)

Example training samples:
 [[0.         0.         0.0251301 ]
 [0.0083767  0.         0.05024432]
 [0.0251248  0.0083767  0.07532681]]
Example training targets: [0.0083767  0.0251248  0.05002503]


# **1.1.4 — RBFNN Implementation (Static Network with K = 16 Neurons)**

In [ ]:


K = 16  # number of RBF neurons

# ---- 1. Compute RBF centers using k-means ----
kmeans = KMeans(n_clusters=K, random_state=0).fit(X_train)
centers = kmeans.cluster_centers_   # shape: (16, 3)

# ---- 2. Compute widths (σ) ----
# Compute all pairwise distances between centers
dists = np.linalg.norm(centers[:, None, :] - centers[None, :, :], axis=2)
sigma = np.max(dists) / np.sqrt(2 * K)

# ---- 3. RBF activation function ----
def rbf(x, c, s):
    return np.exp(-np.linalg.norm(x - c)**2 / (s**2))

# ---- 4. Build Φ matrix for training data ----
Phi_train = np.zeros((X_train.shape[0], K))

for i in range(X_train.shape[0]):
    for k in range(K):
        Phi_train[i, k] = rbf(X_train[i], centers[k], sigma)

# ---- 5. Train output weights using least squares ----
alpha = np.linalg.pinv(Phi_train) @ Y_train
